https://docs.ray.io/en/latest/workflows/index.html

In [1]:
import os
import time
import ray
from ray import workflow
from ray_cluster_control import start_ray_cluster, stop_ray_cluster


Python-dotenv could not parse statement starting at line 4


In [2]:
start_ray_cluster()

RayCluster "michael" has started
Access your cluster dashboard at http://ray-dashboard-michael-open-data-hub.apps.et-gpu.zfq7.p1.openshiftapps.com


In [3]:
stop_ray_cluster()

done


In [ ]:
ray_head = os.environ['RAY_CLUSTER_NAME']
ray.init(address = f"ray://{ray_head}-ray-head:10001")

In [4]:
@ray.remote
def one() -> int:
    return 1

@ray.remote
def add(a: int, b: int) -> int:
    return a + b

dag = add.bind(100, one.bind())

In [5]:
workflow.run(dag, workflow_id="run_1")

2022-10-11 20:15:25,428	INFO workflow_access.py:356 -- Initializing workflow manager...
2022-10-11 20:15:27,131	INFO api.py:203 -- Workflow job created. [id="run_1"].
(WorkflowManagementActor pid=369) 2022-10-11 20:15:27,161	INFO workflow_executor.py:86 -- Workflow job [id=run_1] started.
(WorkflowManagementActor pid=369) 2022-10-11 20:15:27,169	INFO workflow_executor.py:284 -- Task status [SUCCESSFUL]	[run_1@__main__.one]
(_workflow_task_executor_remote pid=400) 2022-10-11 20:15:27,167	INFO task_executor.py:78 -- Task status [RUNNING]	[run_1@__main__.one]


101

(WorkflowManagementActor pid=369) 2022-10-11 20:15:27,175	INFO workflow_executor.py:284 -- Task status [SUCCESSFUL]	[run_1@__main__.add]
(WorkflowManagementActor pid=369) 2022-10-11 20:15:27,177	INFO workflow_executor.py:122 -- Workflow 'run_1' completes successfully.
(_workflow_task_executor_remote pid=400) 2022-10-11 20:15:27,173	INFO task_executor.py:78 -- Task status [RUNNING]	[run_1@__main__.add]


In [6]:
out_ref = workflow.run_async(dag, workflow_id="run_1b")

2022-10-11 20:16:37,070	INFO api.py:203 -- Workflow job created. [id="run_1b"].
(WorkflowManagementActor pid=369) 2022-10-11 20:16:37,094	INFO workflow_executor.py:86 -- Workflow job [id=run_1b] started.
(WorkflowManagementActor pid=369) 2022-10-11 20:16:37,099	INFO workflow_executor.py:284 -- Task status [SUCCESSFUL]	[run_1b@__main__.one]
(WorkflowManagementActor pid=369) 2022-10-11 20:16:37,104	INFO workflow_executor.py:284 -- Task status [SUCCESSFUL]	[run_1b@__main__.add]
(WorkflowManagementActor pid=369) 2022-10-11 20:16:37,118	INFO workflow_executor.py:122 -- Workflow 'run_1b' completes successfully.
(_workflow_task_executor_remote pid=400) 2022-10-11 20:16:37,097	INFO task_executor.py:78 -- Task status [RUNNING]	[run_1b@__main__.one]
(_workflow_task_executor_remote pid=400) 2022-10-11 20:16:37,101	INFO task_executor.py:78 -- Task status [RUNNING]	[run_1b@__main__.add]


In [7]:
ray.get(out_ref)

101

In [8]:
workflow.run(dag, workflow_id="run_2") == 101

2022-10-11 20:17:28,243	INFO api.py:203 -- Workflow job created. [id="run_2"].


True

(WorkflowManagementActor pid=369) 2022-10-11 20:17:28,267	INFO workflow_executor.py:86 -- Workflow job [id=run_2] started.
(WorkflowManagementActor pid=369) 2022-10-11 20:17:28,271	INFO workflow_executor.py:284 -- Task status [SUCCESSFUL]	[run_2@__main__.one]
(WorkflowManagementActor pid=369) 2022-10-11 20:17:28,275	INFO workflow_executor.py:284 -- Task status [SUCCESSFUL]	[run_2@__main__.add]
(WorkflowManagementActor pid=369) 2022-10-11 20:17:28,277	INFO workflow_executor.py:122 -- Workflow 'run_2' completes successfully.
(_workflow_task_executor_remote pid=400) 2022-10-11 20:17:28,269	INFO task_executor.py:78 -- Task status [RUNNING]	[run_2@__main__.one]
(_workflow_task_executor_remote pid=400) 2022-10-11 20:17:28,273	INFO task_executor.py:78 -- Task status [RUNNING]	[run_2@__main__.add]


In [9]:
assert workflow.get_status("run_2") == workflow.WorkflowStatus.SUCCESSFUL
assert workflow.get_output("run_2") == 101
# workflow.get_output_async returns an ObjectRef.
assert ray.get(workflow.get_output_async("run_2")) == 101

In [10]:
# New workflow

In [11]:
@ray.remote
def hello():
    return "Hello"

@ray.remote
def words():
    return [hello.remote(), ray.put("world")]

@ray.remote
def concat(words):
    return " ".join([ray.get(w) for w in words])

In [12]:
workflow.run(concat.bind(words.bind()), workflow_id="run_3")

2022-10-11 20:17:37,871	INFO api.py:203 -- Workflow job created. [id="run_3"].
(WorkflowManagementActor pid=369) 2022-10-11 20:17:37,900	INFO workflow_executor.py:86 -- Workflow job [id=run_3] started.
(_workflow_task_executor_remote pid=400) 2022-10-11 20:17:37,904	INFO task_executor.py:78 -- Task status [RUNNING]	[run_3@__main__.words]


'Hello world'

(WorkflowManagementActor pid=369) 2022-10-11 20:17:39,029	INFO workflow_executor.py:284 -- Task status [SUCCESSFUL]	[run_3@__main__.words]
(WorkflowManagementActor pid=369) 2022-10-11 20:17:39,035	INFO workflow_executor.py:284 -- Task status [SUCCESSFUL]	[run_3@__main__.concat]
(WorkflowManagementActor pid=369) 2022-10-11 20:17:39,037	INFO workflow_executor.py:122 -- Workflow 'run_3' completes successfully.
(_workflow_task_executor_remote pid=400) 2022-10-11 20:17:39,033	INFO task_executor.py:78 -- Task status [RUNNING]	[run_3@__main__.concat]


In [13]:
@ray.remote
def add(a: int, b: int) -> int:
    return a + b

@ray.remote
def fib(n: int) -> int:
    if n <= 1:
        return n
    # return a continuation of a DAG
    return workflow.continuation(add.bind(fib.bind(n - 1), fib.bind(n - 2)))

In [ ]:
# current does not work with clusters https://docs.ray.io/en/latest/workflows/advanced.html
# assert workflow.run(fib.bind(10), workflow_id="run_4") == 55

In [15]:
@ray.remote
def read_data(num):
    return [i for i in range(num)]

@ray.remote
def preprocess(data):
    return [d**2 for d in data]

@ray.remote
def agg(data):
    return sum(data)

In [16]:
data = read_data.bind(10)
preprocessed_data = preprocess.bind(data)
agg_data = agg.bind(preprocessed_data)

In [17]:
#needs https://graphviz.gitlab.io/download/ installed into image.
#ray.dag.vis_utils.plot(agg_data, "output.jpg")

In [18]:
print(workflow.run(agg_data, workflow_id="run_5"))

2022-10-11 20:19:05,279	INFO api.py:203 -- Workflow job created. [id="run_5"].


285


(WorkflowManagementActor pid=369) 2022-10-11 20:19:05,308	INFO workflow_executor.py:86 -- Workflow job [id=run_5] started.
(WorkflowManagementActor pid=369) 2022-10-11 20:19:05,319	INFO workflow_executor.py:284 -- Task status [SUCCESSFUL]	[run_5@__main__.read_data]
(WorkflowManagementActor pid=369) 2022-10-11 20:19:05,324	INFO workflow_executor.py:284 -- Task status [SUCCESSFUL]	[run_5@__main__.preprocess]
(WorkflowManagementActor pid=369) 2022-10-11 20:19:05,328	INFO workflow_executor.py:284 -- Task status [SUCCESSFUL]	[run_5@__main__.agg]
(WorkflowManagementActor pid=369) 2022-10-11 20:19:05,331	INFO workflow_executor.py:122 -- Workflow 'run_5' completes successfully.
(_workflow_task_executor_remote pid=400) 2022-10-11 20:19:05,317	INFO task_executor.py:78 -- Task status [RUNNING]	[run_5@__main__.read_data]
(_workflow_task_executor_remote pid=400) 2022-10-11 20:19:05,322	INFO task_executor.py:78 -- Task status [RUNNING]	[run_5@__main__.preprocess]
(_workflow_task_executor_remote pid=

In [19]:
out_ref = workflow.run_async(agg_data, workflow_id="run_6")

2022-10-11 20:19:07,851	INFO api.py:203 -- Workflow job created. [id="run_6"].
(WorkflowManagementActor pid=369) 2022-10-11 20:19:07,876	INFO workflow_executor.py:86 -- Workflow job [id=run_6] started.
(WorkflowManagementActor pid=369) 2022-10-11 20:19:07,880	INFO workflow_executor.py:284 -- Task status [SUCCESSFUL]	[run_6@__main__.read_data]
(WorkflowManagementActor pid=369) 2022-10-11 20:19:07,884	INFO workflow_executor.py:284 -- Task status [SUCCESSFUL]	[run_6@__main__.preprocess]
(WorkflowManagementActor pid=369) 2022-10-11 20:19:07,888	INFO workflow_executor.py:284 -- Task status [SUCCESSFUL]	[run_6@__main__.agg]
(WorkflowManagementActor pid=369) 2022-10-11 20:19:07,890	INFO workflow_executor.py:122 -- Workflow 'run_6' completes successfully.
(_workflow_task_executor_remote pid=400) 2022-10-11 20:19:07,879	INFO task_executor.py:78 -- Task status [RUNNING]	[run_6@__main__.read_data]
(_workflow_task_executor_remote pid=400) 2022-10-11 20:19:07,883	INFO task_executor.py:78 -- Task st

In [20]:
print(ray.get(out_ref))

285


In [21]:
workflow.get_status(workflow_id="run_6")


<WorkflowStatus.SUCCESSFUL: 'SUCCESSFUL'>

In [22]:
workflow.delete(workflow_id="run_6")

In [23]:
workflow.list_all()

[('run_1', <WorkflowStatus.SUCCESSFUL: 'SUCCESSFUL'>),
 ('run_1b', <WorkflowStatus.SUCCESSFUL: 'SUCCESSFUL'>),
 ('run_2', <WorkflowStatus.SUCCESSFUL: 'SUCCESSFUL'>),
 ('run_3', <WorkflowStatus.SUCCESSFUL: 'SUCCESSFUL'>),
 ('run_5', <WorkflowStatus.SUCCESSFUL: 'SUCCESSFUL'>),
 ('run_4', <WorkflowStatus.FAILED: 'FAILED'>)]

In [24]:
workflow_metadata = workflow.get_metadata("run_5")

In [25]:
workflow_metadata

{'status': 'SUCCESSFUL',
 'user_metadata': {},
 'stats': {'start_time': 1665519545.307062, 'end_time': 1665519545.3309364}}

In [26]:
event1_task = workflow.wait_for_event(workflow.event_listener.TimerListener, time.time()+ 60)
event2_task = workflow.wait_for_event(workflow.event_listener.TimerListener, time.time() +30)

@ray.remote
def gather(*args):
    return args

In [27]:
workflow.run(gather.bind(event1_task,event2_task), workflow_id="gather")

2022-10-11 20:19:32,730	INFO api.py:203 -- Workflow job created. [id="gather"].
(_workflow_task_executor_remote pid=233, ip=10.128.4.36) 2022-10-11 20:19:32,765	INFO task_executor.py:78 -- Task status [RUNNING]	[gather@ray.workflow.api.wait_for_event.locals.get_message_1]
(WorkflowManagementActor pid=369) 2022-10-11 20:19:32,758	INFO workflow_executor.py:86 -- Workflow job [id=gather] started.
(_workflow_task_executor_remote pid=400) 2022-10-11 20:19:32,761	INFO task_executor.py:78 -- Task status [RUNNING]	[gather@ray.workflow.api.wait_for_event.locals.get_message]
(WorkflowManagementActor pid=369) 2022-10-11 20:20:01,774	INFO workflow_executor.py:284 -- Task status [SUCCESSFUL]	[gather@ray.workflow.api.wait_for_event.locals.get_message_1]
(WorkflowManagementActor pid=369) 2022-10-11 20:20:02,820	INFO workflow_executor.py:284 -- Task status [SUCCESSFUL]	[gather@ray.workflow.api.wait_for_event.locals.message_committed_1]
(_workflow_task_executor_remote pid=233, ip=10.128.4.36) 2022-10-1

(None, None)

In [28]:
stop_ray_cluster()

Log channel is reconnecting. Logs produced while the connection was down can be found on the head node of the cluster in `ray_client_server_[port].out`
2022-10-11 20:20:34,287	WARNING dataclient.py:395 -- Encountered connection issues in the data channel. Attempting to reconnect.


done


2022-10-11 20:21:04,502	WARNING dataclient.py:402 -- Failed to reconnect the data channel
